# Deeper Depth Prediction with Fully Convolutional Residual Networks

By [Iro Laina](http://campar.in.tum.de/Main/IroLaina), [Christian Rupprecht](http://campar.in.tum.de/Main/ChristianRupprecht), [Vasileios Belagiannis](http://www.robots.ox.ac.uk/~vb/), [Federico Tombari](http://campar.in.tum.de/Main/FedericoTombari), [Nassir Navab](http://campar.in.tum.de/Main/NassirNavab).

Modified from predict.py

In [1]:
import argparse
import glob, os
import numpy as np
import scipy.ndimage
import tensorflow as tf
from matplotlib import pyplot as plt
from matplotlib import image as im
%matplotlib inline
from PIL import Image
import time
import png

import models

## Loading the model

In [2]:
# Path to pre-trained model
model_data_path = "./NYU_ResNet-UpProj.npy"
# Default input size
height = 228
width = 304
channels = 3
batch_size = 1
# Create a placeholder for the input image
input_node = tf.placeholder(tf.float32, shape=(None, height, width, channels))
    
# Construct the network
net = models.ResNet50UpProj({'data': input_node}, batch_size)
    
sess = tf.Session()
    
# Load the converted parameters
start_time = time.time()
print('Loading the model...')
net.load(model_data_path, sess)
    
uninitialized_vars = []
for var in tf.global_variables():
    try:
        sess.run(var)
    except tf.errors.FailedPreconditionError:
        uninitialized_vars.append(var)
            
init_new_vars_op = tf.variables_initializer(uninitialized_vars)
sess.run(init_new_vars_op)
print('Model is ready')
print("--- Model loading time: %s seconds ---" % (time.time() - start_time))

Loading the model...
Model is ready
--- Model loading time: 142.91447758674622 seconds ---


In [3]:
# save to 16-bit grey-scale
# use pypng
# https://stackoverflow.com/questions/25696615/can-i-save-a-numpy-array-as-a-16-bit-image-using-normal-enthought-python/25814423#25814423
# nparray: 2D numpy array
# file2save: filename with path 
def numpy2grey16(nparray, file2save):
    # Convert to 16 bit unsigned integers.
    z = (65535*((nparray - nparray.min())/nparray.ptp())).astype(np.uint16)
    # Use pypng to write zgray as a grayscale PNG.
    with open(file2save, 'wb') as f:
        writer = png.Writer(width=z.shape[1], height=z.shape[0], bitdepth=16, greyscale=True)
        zgray2list = z.tolist()
        writer.write(f, zgray2list)

## You can skip the following step to the next one if you want to predict series of images
## Predict the single depth images

In [ ]:
start_time = time.time()
print('Reading image')
# Read image
image_path = "./Images/new_office.jpg"
img = Image.open(image_path)
img = img.resize([width,height], Image.ANTIALIAS)
img = np.array(img).astype('float32')
img = np.expand_dims(np.asarray(img), axis = 0)
print("--- Image reading time : %s seconds ---" % (time.time() - start_time))
print('Predicting')
start_time = time.time()
pred = sess.run(net.get_output(), feed_dict={input_node: img})
print(type(pred))
print("--- Predicting time : %s seconds ---" % (time.time() - start_time))

# Plot result
fig = plt.figure()
ii = plt.imshow(pred[0,:,:,0], interpolation='nearest', cmap=plt.cm.get_cmap('Greys_r'))
fig.colorbar(ii)
plt.show()

newpred = np.array(pred[0,:,:,0])
numpy2grey16(newpred, "new22.png")
tmp = Image.open("new22.png")
print(tmp.size)
print(tmp.mode)
print(tmp.getextrema())


(m, predsizey, predsizex, n) = pred.shape
print (m, predsizex, predsizey, n)
upscaled = scipy.ndimage.interpolation.zoom(pred[0,:,:,0], (480./128, 640./160))

im.imsave("test",upscaled16,cmap=plt.cm.get_cmap('Greys_r'))

Predict multiple depth images
------------------------

In [4]:
# windows
seriesPath = ".\\series"
outputPath = ".\\output"
seriesFiles = seriesPath + "\\*.png"
# Linux
# seriesPath = "./series"
# outputPath = "./output"
# seriesFiles = seriesPath + "/*.png"

# Length of path
seriesPathLength = len(seriesPath)
print(seriesPathLength)
start_time = time.time()

# get file name
for file in glob.glob(seriesFiles):
    filename = file[seriesPathLength + 1:]
    print(filename)
    img = Image.open(file)
    img = img.resize([width,height], Image.ANTIALIAS)
    img = np.array(img).astype('float32')
    img = np.expand_dims(np.asarray(img), axis = 0)
    # predict
    pred = sess.run(net.get_output(), feed_dict={input_node: img})
    upscaled = scipy.ndimage.interpolation.zoom(pred[0,:,:,0], (480./128, 640./160))
    # save the depth map
    # Windows
    numpy2grey16(upscaled, outputPath + "\\" + filename)
    # im.imsave(outputPath + "\\" + filename, upscaled, cmap=plt.cm.get_cmap('Greys_r'))
    # Linux
    # im.imsave(outputPath + "/" + filename, upscaled, cmap=plt.cm.get_cmap('Greys_r'))
    print (filename + " saved" + " in" + outputPath)
    
    
print("--- Predicting time : %s seconds ---" % (time.time() - start_time))
    


8
r-1295148646.180126-2906414360.png
r-1295148646.180126-2906414360.png saved in.\output
r-1295148646.210704-2908414648.png
r-1295148646.210704-2908414648.png saved in.\output
r-1295148646.245755-2910414935.png
r-1295148646.245755-2910414935.png saved in.\output
r-1295148646.277860-2912415223.png
r-1295148646.277860-2912415223.png saved in.\output
r-1295148646.309921-2914415510.png
r-1295148646.309921-2914415510.png saved in.\output
r-1295148646.341792-2916415798.png
r-1295148646.341792-2916415798.png saved in.\output
r-1295148646.373948-2918416085.png
r-1295148646.373948-2918416085.png saved in.\output
r-1295148646.405835-2920416373.png
r-1295148646.405835-2920416373.png saved in.\output
r-1295148646.438184-2922416660.png
r-1295148646.438184-2922416660.png saved in.\output
r-1295148646.469876-2924416948.png
r-1295148646.469876-2924416948.png saved in.\output
r-1295148646.534079-2926417235.png
r-1295148646.534079-2926417235.png saved in.\output
r-1295148646.612926-2932418098.png
r-1295

In [5]:
sess.close()